In [39]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,export_graphviz
import graphviz
import dtreeviz
import pandas as pd
import numpy as np
import pickle
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import tree
from pypinyin import pinyin, lazy_pinyin, Style
from autogluon.tabular import TabularDataset, TabularPredictor

In [40]:
# autogluon保存路径
model_path = r"F:\cache_data\zone_ana\dy\modle\autogluon_type_class"
# 检查路径是否存在，否则便创建
if not os.path.exists(model_path):
    os.makedirs(model_path)
else:
    print("文件夹已存在")

文件夹已存在


In [41]:
dataset = pd.read_csv(r"F:\cache_data\zone_ana\dy\train_data\result.csv")

In [42]:
dataset.columns

Index(['FID', 'DLMC', 'new_TZ', 'XMin', 'YMin', 'XMax', 'YMax', 'Centroid_X',
       'Centroid_Y', 'Riverdis',
       ...
       'MRVBF_COUNT', 'MRVBF_AREA', 'MRVBF_MIN', 'MRVBF_MAX', 'MRVBF_RANGE',
       'MRVBF_MEAN', 'MRVBF_STD', 'MRVBF_SUM', 'MRVBF_MEDIAN', 'MRVBF_PCT90'],
      dtype='object', length=401)

In [43]:
dataset["TZ_label"] = dataset.new_TZ.astype("category").cat.codes

In [44]:
dataset['DLMC'] = dataset['DLMC'].astype('category')
dataset['TZ_label'] = dataset['TZ_label'].astype('category')

In [45]:
result = dataset.groupby('TZ_label', observed=True)["new_TZ"].apply(lambda x: list(x.unique())).to_dict()
for one_type in result:
    result[one_type] = result[one_type][0]
print(result)

{0: '中层灰泥质黄色石灰土', 1: '中层砂泥质黄壤', 2: '厚层砂泥质黄壤', 3: '厚层红泥质黄壤', 4: '厚层黏质黄色石灰土', 5: '浅石灰泥田', 6: '浅石红泥田', 7: '潮泥田', 8: '潮砂泥田', 9: '石灰泥田', 10: '石红泥田', 11: '腐中层壤质黄色石灰土', 12: '腐中层壤质黑色石灰土', 13: '腐中层灰泥质黄壤', 14: '腐中层砂泥质黄壤', 15: '腐中层砂泥质黄壤性土', 16: '腐中层砾壤质酸性紫色土', 17: '腐中层砾壤质钙质紫色土', 18: '腐中层硅质黄壤', 19: '腐中层红泥质黄壤', 20: '腐厚层壤质黄色石灰土', 21: '腐厚层壤质黑色石灰土', 22: '腐厚层灰泥质黄壤', 23: '腐厚层砂泥质黄壤', 24: '腐厚层硅质黄壤', 25: '腐厚层红泥质黄壤', 26: '腐薄层壤质黄色石灰土', 27: '腐薄层灰泥质黄壤', 28: '腐薄层砂泥质黄壤', 29: '腐薄层砾砂质黑色石灰土', 30: '腐薄层硅质山地灌丛草甸土', 31: '腐薄层硅质黄壤', 32: '腐薄层红泥质黄壤', 33: '腐薄层黏质黄色石灰土', 34: '薄层壤质中性紫色土', 35: '薄层灰泥质黄色石灰土', 36: '薄层砂泥质黄壤', 37: '薄层砾壤质中性紫色土', 38: '薄层砾壤质黄色石灰土', 39: '薄层砾壤质黑色石灰土', 40: '薄层砾硅质黄壤', 41: '薄层硅质酸性粗骨土', 42: '轻漂白粉泥田', 43: '重漂砂泥田', 44: '重漂红泥田', 45: '青石灰泥田', 46: '黄浅白粉泥田', 47: '黄浅砂泥田', 48: '黄浅红泥田', 49: '黄浅鳝泥田', 50: '黄白粉泥田', 51: '黄砂泥田', 52: '黄红泥田', 53: '黄青白粉泥田', 54: '黄鳝泥田', 55: '黄黄砂泥田'}


In [46]:
# 保存映射关系
with open(r'D:\worker_code\Terrain_Test\data\soil_dict.json', 'w') as f:
    json.dump(result, f)

In [ ]:
# 筛选水稻土数据
sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂'])]

In [10]:
pd.unique(sdt_data['new_TZ'])

array(['黄浅白粉泥田', '黄鳝泥田', '浅石红泥田', '石红泥田', '潮砂泥田', '石灰泥田', '黄红泥田', '青石灰泥田',
       '黄黄砂泥田', '轻漂白粉泥田', '黄青白粉泥田', '黄浅红泥田', '黄白粉泥田', '黄浅砂泥田', '重漂红泥田',
       '潮泥田', '浅石灰泥田', '黄砂泥田', '黄浅鳝泥田', '重漂砂泥田'], dtype=object)

In [11]:
# 筛选非自然土数据
fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂'])]

In [12]:
pd.unique(fzrt_data['new_TZ']),fzrt_data.shape

(array(['薄层砾硅质黄壤', '厚层砂泥质黄壤', '中层灰泥质黄色石灰土', '厚层红泥质黄壤', '中层砂泥质黄壤',
        '薄层硅质酸性粗骨土', '薄层灰泥质黄色石灰土', '厚层黏质黄色石灰土', '薄层壤质中性紫色土', '薄层砾壤质黑色石灰土',
        '薄层砂泥质黄壤', '薄层砾壤质黄色石灰土', '薄层砾壤质中性紫色土'], dtype=object),
 (1075, 402))

In [13]:
# 筛选自然土数据
zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])]

In [14]:
zrt_data.shape

(27012, 402)

In [15]:
pd.unique(zrt_data['new_TZ'])

array(['腐厚层砂泥质黄壤', '腐薄层灰泥质黄壤', '腐中层砂泥质黄壤', '腐中层壤质黄色石灰土', '腐薄层硅质黄壤',
       '腐薄层硅质山地灌丛草甸土', '腐中层硅质黄壤', '腐薄层砂泥质黄壤', '腐薄层壤质黄色石灰土', '腐薄层砾砂质黑色石灰土',
       '腐厚层硅质黄壤', '腐中层壤质黑色石灰土', '腐厚层红泥质黄壤', '腐薄层黏质黄色石灰土', '腐厚层灰泥质黄壤',
       '腐中层红泥质黄壤', '腐中层灰泥质黄壤', '腐厚层壤质黄色石灰土', '腐中层砾壤质钙质紫色土', '腐中层砾壤质酸性紫色土',
       '腐薄层红泥质黄壤', '腐中层砂泥质黄壤性土', '腐厚层壤质黑色石灰土'], dtype=object)

In [16]:
sdt_data.shape,fzrt_data.shape,zrt_data.shape,dataset.shape,sdt_data.shape[0]+fzrt_data.shape[0]+zrt_data.shape[0]==dataset.shape[0]

((4058, 402), (1075, 402), (27012, 402), (32145, 402), True)

In [17]:
# 定义要检查的关键字列表
keywords = ['DEM','ndvi', 'PCA_0', 'Slope','Aspect', 'MRVBF','TopographicWetnessIndex', 'Mean',  'PH','PRE','SRA','TMP','VAP','WIND','PlanCurvature']

# 定义包含字符串的列表
strings = list(dataset.columns)

# 使用列表推导式和any函数检查并添加到新列表
filtered_strings = [s for s in strings if any(keyword in s for keyword in keywords)]
print(len(filtered_strings))
print(filtered_strings)


160
['DEM_COUNT', 'DEM_AREA', 'DEM_MIN', 'DEM_MAX', 'DEM_RANGE', 'DEM_MEAN', 'DEM_STD', 'DEM_SUM', 'DEM_MEDIAN', 'DEM_PCT90', 'Aspect_COUNT', 'Aspect_AREA', 'Aspect_MIN', 'Aspect_MAX', 'Aspect_RANGE', 'Aspect_MEAN', 'Aspect_STD', 'Aspect_SUM', 'Aspect_MEDIAN', 'Aspect_PCT90', 'PlanCurvature_COUNT', 'PlanCurvature_AREA', 'PlanCurvature_MIN', 'PlanCurvature_MAX', 'PlanCurvature_RANGE', 'PlanCurvature_MEAN', 'PlanCurvature_STD', 'PlanCurvature_SUM', 'PlanCurvature_MEDIAN', 'PlanCurvature_PCT90', 'RelativeSlopePosition_COUNT', 'RelativeSlopePosition_AREA', 'RelativeSlopePosition_MIN', 'RelativeSlopePosition_MAX', 'RelativeSlopePosition_RANGE', 'RelativeSlopePosition_MEAN', 'RelativeSlopePosition_STD', 'RelativeSlopePosition_SUM', 'RelativeSlopePosition_MEDIAN', 'RelativeSlopePosition_PCT90', 'Slope_COUNT', 'Slope_AREA', 'Slope_MIN', 'Slope_MAX', 'Slope_RANGE', 'Slope_MEAN', 'Slope_STD', 'Slope_SUM', 'Slope_MEDIAN', 'Slope_PCT90', 'TopographicWetnessIndex_COUNT', 'TopographicWetnessIndex_AR

In [18]:
# features = ['DEM', 'Mean', 'ndvi', 'PCA_0', 'LON', 'LAT', 'PH','DL','DZ']
# features = list(dataset.columns[1:-1])
sdt_features= ['DLMC','XMin','XMax','YMin','YMax','Centroid_X','Centroid_Y','Riverdis',
           'DEM_MAX','DEM_MIN','DEM_MEAN','DEM_RANGE','DEM_AREA',
           'ndvi_MEAN',
           'PCA_0_MEAN',
           'Slope_MAX', 'Slope_MIN','Slope_MEAN','Slope_RANGE',
           'Aspect_MEAN',
           'MRVBF_MAX','MRVBF_MIN','MRVBF_MEAN','MRVBF_RANGE',
           'TopographicWetnessIndex_MEAN','TopographicWetnessIndex_RANGE',
           'Mean_MEAN',
           'PH_MEAN',
           'PRE_MEAN','PRE_RANGE',
           'SRA_MEAN','SRA_RANGE',
           'TMP_MEAN','TMP_RANGE',
           'VAP_MEAN','VAP_RANGE',
           'WIND_MEAN','WIND_RANGE',
           'PlanCurvature_MEAN','PlanCurvature_RANGE'
         ]
fzrt_features = ['DLMC','XMin','XMax','YMin','YMax','Centroid_X','Centroid_Y',
           'DEM_MAX','DEM_MIN','DEM_MEAN','DEM_RANGE','DEM_AREA',
           'ndvi_MEAN',
           'PCA_0_MEAN',
           'Slope_MAX', 'Slope_MIN','Slope_MEAN','Slope_RANGE',
           'Aspect_MEAN',
           'MRVBF_MAX','MRVBF_MIN','MRVBF_MEAN','MRVBF_RANGE',
           'TopographicWetnessIndex_MEAN','TopographicWetnessIndex_RANGE',
           'Mean_MEAN',
           'PH_MEAN',
           'PRE_MEAN','PRE_RANGE',
           'SRA_MEAN','SRA_RANGE',
           'TMP_MEAN','TMP_RANGE',
           'VAP_MEAN','VAP_RANGE',
           'WIND_MEAN','WIND_RANGE',
           'PlanCurvature_MEAN','PlanCurvature_RANGE'
         ]
zrt_features = ['DLMC','XMin','XMax','YMin','YMax','Centroid_X','Centroid_Y',
           'DEM_MAX','DEM_MIN','DEM_MEAN','DEM_RANGE','DEM_AREA',
           'ndvi_MEAN',
           'PCA_0_MEAN',
           'Slope_MAX', 'Slope_MIN','Slope_MEAN','Slope_RANGE',
           'Aspect_MEAN',
           'MRVBF_MAX','MRVBF_MIN','MRVBF_MEAN','MRVBF_RANGE',
           'TopographicWetnessIndex_MEAN','TopographicWetnessIndex_RANGE',
           'Mean_MEAN',
           'PH_MEAN',
           'PRE_MEAN','PRE_RANGE',
           'SRA_MEAN','SRA_RANGE',
           'TMP_MEAN','TMP_RANGE',
           'VAP_MEAN','VAP_RANGE',
           'WIND_MEAN','WIND_RANGE',
           'PlanCurvature_MEAN','PlanCurvature_RANGE'
         ]
target = "TZ_label"

In [19]:
# 获取sdt数据集
sdt_data = sdt_data[sdt_features+[target]]

In [20]:
# 获取非自然土数据集
fzrt_data = fzrt_data[fzrt_features+[target]]

In [21]:
# 获取自然土数据集
zrt_data = zrt_data[zrt_features+[target]]

In [22]:
# 分割训练集和测试集
sdt_data_train,sdt_data_test = train_test_split(sdt_data, test_size=0.2, random_state=42)
fzrt_data_train,fzrt_data_test = train_test_split(fzrt_data, test_size=0.2, random_state=42)
zrt_data_train,zrt_data_test = train_test_split(zrt_data, test_size=0.2, random_state=42)

# data = data
label = "TZ_label"

In [23]:

hyperparameters={
	'NN_TORCH': {},
	'FASTAI': {},
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}]
}


In [24]:
# 训练水稻土模型
sdt_model_path = os.path.join(model_path,'sdt_model')
# 检查路径是否存在，否则便创建
if not os.path.exists(sdt_model_path):
    os.makedirs(sdt_model_path)
else:
    print("文件夹已存在")
sdt_predictor = TabularPredictor(label=label,path=sdt_model_path,eval_metric="f1_weighted",problem_type='multiclass').fit(sdt_data_train,hyperparameters=hyperparameters)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\dy\modle\autogluon_type_class\sdt_model"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       45.63 GB / 63.81 GB (71.5%)
Disk Space Avail:   475

文件夹已存在


	0.6469	 = Validation score   (f1_weighted)
	2.93s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.8491	 = Validation score   (f1_weighted)
	0.65s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.8488	 = Validation score   (f1_weighted)
	0.78s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.8311	 = Validation score   (f1_weighted)
	0.57s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.8442	 = Validation score   (f1_weighted)
	0.6s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.5851	 = Validation score   (f1_weighted)
	8.99s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'RandomForestGini': 0.348, 'RandomForestEntr': 0.275, 'ExtraTreesGini': 0.174, 'ExtraTreesEntr': 0.174, 'NeuralNetFastAI': 0.014, 'NeuralNetTorch': 0.014}
	0

In [25]:
# 获取最佳模型
sdt_best_model = sdt_predictor.model_best
# 获取WeightedEnsemble_L3模型
# 获取模型的训练摘要
sdt_model_summary = sdt_predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.864259  f1_weighted       0.152302  15.048771                0.000000           0.529552            2       True          7
1     RandomForestGini   0.849114  f1_weighted       0.038327   0.646397                0.038327           0.646397            1       True          2
2     RandomForestEntr   0.848837  f1_weighted       0.039677   0.780836                0.039677           0.780836            1       True          3
3       ExtraTreesEntr   0.844162  f1_weighted       0.035932   0.596082                0.035932           0.596082            1       True          5
4       ExtraTreesGini   0.831065  f1_weighted       0.031255   0.569609                0.031255           0.569609            1       True          4
5      NeuralNetFastAI   0.64691

In [26]:
sdt_predictor.leaderboard(sdt_data_train, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.970303,0.864259,f1_weighted,0.468863,0.152302,15.048771,0.000000,0.000000,0.529552,2,True,7
1,RandomForestGini,0.967857,0.849114,f1_weighted,0.086984,0.038327,0.646397,0.086984,0.038327,0.646397,1,True,2
2,RandomForestEntr,0.967846,0.848837,f1_weighted,0.101354,0.039677,0.780836,0.101354,0.039677,0.780836,1,True,3
3,ExtraTreesEntr,0.967520,0.844162,f1_weighted,0.116344,0.035932,0.596082,0.116344,0.035932,0.596082,1,True,5
4,ExtraTreesGini,0.965675,0.831065,f1_weighted,0.115578,0.031255,0.569609,0.115578,0.031255,0.569609,1,True,4
5,NeuralNetFastAI,0.827096,0.646913,f1_weighted,0.027678,0.000000,2.932251,0.027678,0.000000,2.932251,1,True,1
6,NeuralNetTorch,0.654184,0.585140,f1_weighted,0.020924,0.007111,8.994044,0.020924,0.007111,8.994044,1,True,6


In [27]:
sdt_predictor.leaderboard(sdt_data_test,silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr,0.841058,0.848837,f1_weighted,0.082486,0.039677,0.780836,0.082486,0.039677,0.780836,1,True,3
1,WeightedEnsemble_L2,0.838596,0.864259,f1_weighted,0.376107,0.152302,15.048771,0.000000,0.000000,0.529552,2,True,7
2,RandomForestGini,0.833331,0.849114,f1_weighted,0.075530,0.038327,0.646397,0.075530,0.038327,0.646397,1,True,2
3,ExtraTreesEntr,0.825722,0.844162,f1_weighted,0.083620,0.035932,0.596082,0.083620,0.035932,0.596082,1,True,5
4,ExtraTreesGini,0.822682,0.831065,f1_weighted,0.100178,0.031255,0.569609,0.100178,0.031255,0.569609,1,True,4
5,NeuralNetFastAI,0.603495,0.646913,f1_weighted,0.016577,0.000000,2.932251,0.016577,0.000000,2.932251,1,True,1
6,NeuralNetTorch,0.525144,0.585140,f1_weighted,0.017716,0.007111,8.994044,0.017716,0.007111,8.994044,1,True,6


In [28]:
# 训练非自然土模型
fzrt_model_path = os.path.join(model_path,'fzrt_model')
# 检查路径是否存在，否则便创建
if not os.path.exists(fzrt_model_path):
    os.makedirs(fzrt_model_path)
else:
    print("文件夹已存在")
fzrt_predictor = TabularPredictor(label=label,path=fzrt_model_path,eval_metric="f1_weighted",problem_type='multiclass').fit(fzrt_data_train,hyperparameters=hyperparameters)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\dy\modle\autogluon_type_class\fzrt_model"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       45.44 GB / 63.81 GB (71.2%)
Disk Space Avail:   47

文件夹已存在


	0.5899	 = Validation score   (f1_weighted)
	0.52s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7644	 = Validation score   (f1_weighted)
	0.48s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.8091	 = Validation score   (f1_weighted)
	0.5s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7917	 = Validation score   (f1_weighted)
	0.48s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.8068	 = Validation score   (f1_weighted)
	0.51s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.5741	 = Validation score   (f1_weighted)
	5.58s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'RandomForestEntr': 0.857, 'NeuralNetFastAI': 0.071, 'RandomForestGini': 0.071}
	0.8196	 = Validation score   (f1_weighted)
	0.47s	 = Training   runtime
	0.0

In [29]:
# 获取最佳模型
fzrt_best_model = fzrt_predictor.model_best
# 获取WeightedEnsemble_L3模型
# 获取模型的训练摘要
fzrt_model_summary = fzrt_predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.819583  f1_weighted       0.073590  1.967283                0.000000           0.468019            2       True          7
1     RandomForestEntr   0.809070  f1_weighted       0.034384  0.496706                0.034384           0.496706            1       True          3
2       ExtraTreesEntr   0.806839  f1_weighted       0.033601  0.511451                0.033601           0.511451            1       True          5
3       ExtraTreesGini   0.791664  f1_weighted       0.034245  0.480338                0.034245           0.480338            1       True          4
4     RandomForestGini   0.764398  f1_weighted       0.034263  0.482421                0.034263           0.482421            1       True          2
5      NeuralNetFastAI   0.589923  f1_

In [30]:
fzrt_predictor.leaderboard(fzrt_data_train, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.954041,0.819583,f1_weighted,0.116035,0.073590,1.967283,0.000000,0.000000,0.468019,2,True,7
1,RandomForestEntr,0.951818,0.809070,f1_weighted,0.049184,0.034384,0.496706,0.049184,0.034384,0.496706,1,True,3
2,ExtraTreesEntr,0.951647,0.806839,f1_weighted,0.053054,0.033601,0.511451,0.053054,0.033601,0.511451,1,True,5
3,ExtraTreesGini,0.948359,0.791664,f1_weighted,0.049100,0.034245,0.480338,0.049100,0.034245,0.480338,1,True,4
4,RandomForestGini,0.943618,0.764398,f1_weighted,0.055733,0.034263,0.482421,0.055733,0.034263,0.482421,1,True,2
5,NeuralNetFastAI,0.860285,0.589923,f1_weighted,0.011118,0.004943,0.520137,0.011118,0.004943,0.520137,1,True,1
6,NeuralNetTorch,0.830077,0.574134,f1_weighted,0.015457,0.007998,5.578221,0.015457,0.007998,5.578221,1,True,6


In [31]:
fzrt_predictor.leaderboard(fzrt_data_test,silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.803773,0.819583,f1_weighted,0.099297,0.073590,1.967283,0.000000,0.000000,0.468019,2,True,7
1,RandomForestEntr,0.792116,0.809070,f1_weighted,0.050063,0.034384,0.496706,0.050063,0.034384,0.496706,1,True,3
2,RandomForestGini,0.784807,0.764398,f1_weighted,0.044675,0.034263,0.482421,0.044675,0.034263,0.482421,1,True,2
3,ExtraTreesGini,0.763291,0.791664,f1_weighted,0.049125,0.034245,0.480338,0.049125,0.034245,0.480338,1,True,4
4,ExtraTreesEntr,0.749638,0.806839,f1_weighted,0.049188,0.033601,0.511451,0.049188,0.033601,0.511451,1,True,5
5,NeuralNetFastAI,0.600095,0.589923,f1_weighted,0.004559,0.004943,0.520137,0.004559,0.004943,0.520137,1,True,1
6,NeuralNetTorch,0.577058,0.574134,f1_weighted,0.003178,0.007998,5.578221,0.003178,0.007998,5.578221,1,True,6


In [32]:
# 训练自然土模型
zrt_model_path = os.path.join(model_path,'zrt_model')
# 检查路径是否存在，否则便创建
if not os.path.exists(zrt_model_path):
    os.makedirs(zrt_model_path)
else:
    print("文件夹已存在")
zrt_predictor = TabularPredictor(label=label,path=zrt_model_path,eval_metric="f1_weighted",problem_type='multiclass').fit(zrt_data_train,hyperparameters=hyperparameters)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\dy\modle\autogluon_type_class\zrt_model"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       45.37 GB / 63.81 GB (71.1%)
Disk Space Avail:   475

文件夹已存在


	0.8525	 = Validation score   (f1_weighted)
	13.22s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9577	 = Validation score   (f1_weighted)
	2.28s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9615	 = Validation score   (f1_weighted)
	4.6s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9477	 = Validation score   (f1_weighted)
	0.85s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.952	 = Validation score   (f1_weighted)
	0.9s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.8868	 = Validation score   (f1_weighted)
	141.33s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'ExtraTreesEntr': 0.516, 'RandomForestEntr': 0.419, 'RandomForestGini': 0.032, 'ExtraTreesGini': 0.032}
	0.9633	 = Validation score   (f1_weighted)
	0.99s	

In [33]:
# 获取最佳模型
zrt_best_model = zrt_predictor.model_best
# 获取WeightedEnsemble_L3模型
# 获取模型的训练摘要
zrt_model_summary = zrt_predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.963292  f1_weighted       0.203274    9.613321                0.002420           0.992728            2       True          7
1     RandomForestEntr   0.961523  f1_weighted       0.052431    4.596302                0.052431           4.596302            1       True          3
2     RandomForestGini   0.957727  f1_weighted       0.047955    2.283661                0.047955           2.283661            1       True          2
3       ExtraTreesEntr   0.952006  f1_weighted       0.047700    0.895353                0.047700           0.895353            1       True          5
4       ExtraTreesGini   0.947731  f1_weighted       0.052768    0.845278                0.052768           0.845278            1       True          4
5       NeuralNetTorch   0

In [34]:
zrt_predictor.leaderboard(zrt_data_train, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.995789,0.963292,f1_weighted,1.550709,0.203274,9.613321,0.017293,0.002420,0.992728,2,True,7
1,RandomForestEntr,0.995604,0.961523,f1_weighted,0.282958,0.052431,4.596302,0.282958,0.052431,4.596302,1,True,3
2,RandomForestGini,0.995233,0.957727,f1_weighted,0.333272,0.047955,2.283661,0.333272,0.047955,2.283661,1,True,2
3,ExtraTreesEntr,0.994676,0.952006,f1_weighted,0.483035,0.047700,0.895353,0.483035,0.047700,0.895353,1,True,5
4,ExtraTreesGini,0.994259,0.947731,f1_weighted,0.434151,0.052768,0.845278,0.434151,0.052768,0.845278,1,True,4
5,NeuralNetTorch,0.930434,0.886761,f1_weighted,0.100274,0.015664,141.326212,0.100274,0.015664,141.326212,1,True,6
6,NeuralNetFastAI,0.898663,0.852524,f1_weighted,0.148975,0.020249,13.220450,0.148975,0.020249,13.220450,1,True,1


In [35]:
zrt_predictor.leaderboard(zrt_data_test,silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr,0.955300,0.961523,f1_weighted,0.166692,0.052431,4.596302,0.166692,0.052431,4.596302,1,True,3
1,RandomForestGini,0.954304,0.957727,f1_weighted,0.197050,0.047955,2.283661,0.197050,0.047955,2.283661,1,True,2
2,WeightedEnsemble_L2,0.951806,0.963292,f1_weighted,0.998822,0.203274,9.613321,0.001668,0.002420,0.992728,2,True,7
3,ExtraTreesEntr,0.944618,0.952006,f1_weighted,0.316777,0.047700,0.895353,0.316777,0.047700,0.895353,1,True,5
4,ExtraTreesGini,0.941234,0.947731,f1_weighted,0.316635,0.052768,0.845278,0.316635,0.052768,0.845278,1,True,4
5,NeuralNetTorch,0.880660,0.886761,f1_weighted,0.051873,0.015664,141.326212,0.051873,0.015664,141.326212,1,True,6
6,NeuralNetFastAI,0.854673,0.852524,f1_weighted,0.049300,0.020249,13.220450,0.049300,0.020249,13.220450,1,True,1


In [36]:
# 获取sdt模型特征重要性
sdt_feature_importance = sdt_predictor.feature_importance(data=sdt_data_train)
pd.DataFrame(sdt_feature_importance)

Computing feature importance via permutation shuffling for 40 features using 3225 rows with 5 shuffle sets...
	97.05s	= Expected runtime (19.41s per shuffle set)
	27.5s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
XMax,0.010557,0.000921,6.868165e-06,5,0.012452,0.008661
XMin,0.010248,0.000926,7.927353e-06,5,0.012156,0.008341
Centroid_X,0.010062,0.001028,1.291060e-05,5,0.012180,0.007945
YMax,0.007316,0.001631,2.775479e-04,5,0.010673,0.003958
Centroid_Y,0.006944,0.001912,6.247726e-04,5,0.010881,0.003008
YMin,0.006513,0.001413,2.497030e-04,5,0.009421,0.003604
SRA_MEAN,0.006314,0.000808,3.144624e-05,5,0.007977,0.004651
DEM_MEAN,0.006043,0.000415,2.656674e-06,5,0.006898,0.005188
DEM_MIN,0.005860,0.000130,2.940144e-08,5,0.006129,0.005592
PRE_MEAN,0.005494,0.000652,2.333313e-05,5,0.006835,0.004152


In [37]:
# 获取fzrt模型特征重要性
fzrt_feature_importance = fzrt_predictor.feature_importance(data=fzrt_data_train)
pd.DataFrame(fzrt_feature_importance)

Computing feature importance via permutation shuffling for 39 features using 853 rows with 5 shuffle sets...
	22.26s	= Expected runtime (4.45s per shuffle set)
	3.11s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
TMP_MEAN,0.007770,0.003619,0.004321,5,0.015221,0.000319
YMin,0.007609,0.001769,0.000326,5,0.011250,0.003967
Centroid_Y,0.006494,0.002544,0.002328,5,0.011732,0.001257
VAP_MEAN,0.006374,0.000634,0.000012,5,0.007679,0.005069
SRA_MEAN,0.005880,0.003323,0.008364,5,0.012722,-0.000963
XMax,0.005701,0.002303,0.002602,5,0.010443,0.000960
YMax,0.005689,0.001294,0.000300,5,0.008354,0.003025
Centroid_X,0.004966,0.001562,0.001034,5,0.008181,0.001750
XMin,0.004965,0.001315,0.000539,5,0.007673,0.002258
WIND_MEAN,0.004949,0.001759,0.001630,5,0.008571,0.001328


In [38]:
# 获取zrt模型特征重要性
zrt_feature_importance = zrt_predictor.feature_importance(data=zrt_data_train)
pd.DataFrame(zrt_feature_importance)

Computing feature importance via permutation shuffling for 39 features using 5000 rows with 5 shuffle sets...
	215.42s	= Expected runtime (43.08s per shuffle set)
	49.13s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
XMax,1.278686e-02,0.002158,0.000094,5,0.017230,0.008344
XMin,8.970120e-03,0.002381,0.000544,5,0.013873,0.004067
Centroid_X,8.290781e-03,0.001177,0.000048,5,0.010715,0.005867
YMin,7.031791e-03,0.001064,0.000061,5,0.009223,0.004841
Centroid_Y,6.962690e-03,0.001139,0.000083,5,0.009308,0.004617
YMax,6.516546e-03,0.001055,0.000080,5,0.008689,0.004345
SRA_MEAN,3.772447e-03,0.000786,0.000213,5,0.005390,0.002155
PRE_MEAN,2.131172e-03,0.000534,0.000437,5,0.003232,0.001031
WIND_MEAN,1.807959e-03,0.000636,0.001568,5,0.003117,0.000499
TMP_MEAN,1.648734e-03,0.000574,0.001512,5,0.002831,0.000467
